# Theis pumping test - Fitting Theis to measured data
This notebook demonstrate the application of the Theis principle for pumping test evaluation in confined, transient setups. Parts of the notebook use code from Randy Hunt (refer to P3_4 from the textbook of Anderson, Woessner, and Hunt (2015) - available on GITHUB)

## Introduction

### General situation
We consider a confined aquifer with constant transmissivity. If a well is pumping water out of the aquifer, radial flow towards the well is induced. The following figure illustrates this situation.

<img src="FILES\Flow_well_confined.png" width="400">

*Fig: Radial flow in a confined aquifer (Fetter, 1994)*

The calculate the hydraulic situation, the following simplified flow equation can be used. This equation accounts for 1D radial transient flow towards a fully penetrating well within a confined aquifer without further sinks and sources:

$$ \frac{\partial^2 h}{\partial r^2}+\frac{1}{r}\frac{\partial h}{\partial r}=\frac{S}{T}\frac{\partial h}{\partial t} $$


### Solution by Theis
Charles V. Theis presented a solution for this by deriving

$$ s(r,t)=\frac{Q}{4\pi T}W(u) $$

with the well function

$$ W(u) = \int_{u }^{+\infty} \frac{e^{-\tilde u}}{\tilde u}d\tilde u $$

and the dimensionless variable 

$$ u = \frac{Sr^2}{4Tt} $$

This equations are not easy to solve. Historically, values for the well function were provided by tables or as so called type-curve. The type-curve matching with experimental data for pumping test analysis can be considered as one of the basic hydrogeological methods.

However, modern computer provide an easier and more convinient way to solve the 1D radial flow equation based on the Theis approach.

### Computation

Subsequently, the computation is done by general mathematical routines. It is not necessary to understand all details (although the code reveals them). But the application of those methods for pumping test data is of great practical importance.

- Subsequently, parts that are not essential for understanding are marked with *(italic text in brackets)*. You need to run those cells but you don't necessarily need read the code.

- Necessary parts come in **bold text**.

*(First, the necessary Python functions are initiated.)*

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
from ipywidgets import *

**The following section stores the measured data.**

This are two lists with time (in minutes) and drawdown (in meters). Make sure that those elements match each other (e.g. the first time is associated with the first drawdown value etc.)

In [13]:
m_time = [0.50,1.00,1.33,2.00,3.00,5.00,7.00,9.33,13.50,22.00,30.00] # time in minutes
m_ddown = [0.30,0.37,0.38,0.40,0.43,0.49,0.52,0.53,0.55,0.56,0.57]   # drawdown in meters

*(Here the necessary functions like the well function $W(u)$ are defined. Later, those functions are used in the computation)*

In [14]:
#Define a function, class, and object for Theis Well analysis

def well_function(u):
    return scipy.special.exp1(u)

def theis_u(T,S,r,t):
    u = r ** 2 * S / 4. / T / t
    return u

def theis_s(Q, T, u):
    s = Q / 4. / np.pi / T * well_function(u)
    return s

def theis_wu(Q, T, s):
    wu = s * 4. * np.pi * T / Q
    return wu

*(Here, the methode computes the data for the well function. Those data can be used to generate a type curve.)*

In [15]:
u_max = 1
r_max = 100000
u  = [u_max for x in range(r_max)]
um = [u_max for x in range(r_max)]
u_inv  = [r_max/u_max for x in range(r_max)]
um_inv = [r_max/u_max for x in range(r_max)]
w_u  = [well_function(u_max/r_max) for x in range(r_max)]
w_um = [well_function(u_max/r_max) for x in range(r_max)]

for x in range(1,r_max,1):
    if x>0:
        u[x] = x*u_max/r_max
        u_inv[x] = 1/u[x]
        w_u[x] = well_function(u[x])

**Subsequently, the given data for the processing of measured data are provided.**

This are the given data like
- pumping rate $Q$,
- aquifer thickness $b$,
- distance of the observation (where time and drawdown were measured) to the pumping well $r$.

**These data can be adjusted for individual measurements.**

In [17]:
# Parameters needed to solve Theis
r = 23       # m
b = 8.5      # m
Q = 0.0267   #m^3/d

**Here the data for curve fitting are adjusted**

**The pumping test evaluation is more or less the calibration of a model (i.e. the Theis equations) by adjusting parameters. Those parameters include**
- **Hydraulic conductivity** of the entire aquifer $K$
- **Specific storage** of the aquifer $S_S$.

Accordingly, Transmissivity $T$ and Storativity $S$ are computed.

In [18]:
def Theis_plot(T,S):

    # Parameters needed to solve Theis
    #T = 3E-2    # m2/s
    #S = 0.003  # dimensionless

    # Compute K and SS to provide parameters for plausability check
    # (i.e. are the parameter in a reasonable range)
    K = T/b     # m/s
    SS = S/b    # 1/m

    x = 0
    for t in m_time:
        um[x] = theis_u(T,S,r,t*60)
        um_inv[x] = 1/um[x]
        w_um[x] = theis_wu(Q,T,m_ddown[x])
        x = x+1
    
    fig, ax = plt.subplots()
    ax.plot(u_inv, w_u)
    ax.plot(um_inv, w_um,'ro')
    plt.yscale("log")
    plt.xscale("log")
    plt.axis([1,1E5,1E-1,1E+1])
    ax.set(xlabel='1/u', ylabel='w(u)',title='Theis drawdown')
    ax.grid(which="both")
    plt.legend(('well function','measured'))
    plt.show()

    print("Transmissivity  T = K * b = ",T, " m^2/s")
    print("Storativity    S = Ss * b = ",S,"[-]")
    print("Hydraulic conductivity K  =  ","% 10.2E"% K, " m/s")
    print("Specific storage       Ss = ","% 10.3E"%SS, " 1/m")
    print("Aquifer thickness       b = ",b, " m")
    
interact(Theis_plot,
         T=widgets.FloatLogSlider(value=0.03,base=10,min=-9, max=1, step=0.1,readout=True,readout_format='.2e'),
         S=widgets.FloatLogSlider(value=0.003,base=10,min=-9, max=1, step=0.1,readout=True,readout_format='.2e'))

interactive(children=(FloatLogSlider(value=0.03, description='T', max=1.0, min=-9.0, readout_format='.2e'), Fl…

<function __main__.Theis_plot(T, S)>

*(Compute measured data and plot the well function)*

**Plot the match between type curve and measured data**

See what happens and modify the parameters $K$ and $S$. Modify until the fit is fine. 